<a href="https://colab.research.google.com/github/usrmaia/Mundo-de-Wumpus/blob/main/Mundo_de_Wumpus_Refatorado_(ou_quase).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [19]:
from random import random, randint, choice
!pip install tabulate
from tabulate import tabulate
!pip install colorama
from colorama import Fore, init, deinit
!pip install python-sat
from pysat.solvers import Glucose3
!pip install db-sqlite3
import sqlite3
from random import random, choice

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Objetos Úteis

In [20]:
class PreferenciasDeMapa():
  def __init__(self, linhas, colunas, qtd_wumpus, qtd_pocos, qtd_ouros):
    self.linhas = linhas
    self.colunas = colunas
    self.qtd_wumpus = qtd_wumpus
    self.qtd_pocos = qtd_pocos
    self.qtd_ouros = qtd_ouros
  
  def tamanho(self): return (self.linhas, self.colunas)
  
class InfoDoAgente():
  def __init__(self, linha = 0, coluna = 0, orientacao = 0,
              desempenho = 0, status = "explorando"):
    self.linha = linha
    self.coluna = coluna
    self.orientacao = orientacao
    self.desempenho = desempenho
    self.status = status
  
  def posicaoAtual(self): return (self.linha, self.coluna)

  def posicaoAFrente(self):
    linha = self.linha 
    coluna = self.coluna

    if self.orientacao == 0: coluna += 1
    elif self.orientacao == 90: linha -= 1
    elif self.orientacao == 180: coluna -= 1
    elif self.orientacao == 270: linha += 1

    return linha, coluna

  def incrementarOrientacao(self):
    self.orientacao += 90
    if self.orientacao == 360: self.orientacao = 0
  
  def decrementarOrientacao(self):
    self.orientacao -= 90
    if self.orientacao == -90: self.orientacao = 270

class Pilha():
  def __init__(self):
    self.p = []
  
  def pop(self): return self.p.pop()
  def get(self): return self.p
  def len(self): return len(self.p)
  
  def ultPosicao(self):
    if self.p: return self.p[-1]
    return []
  
  def add(self, pos_atual):
    if self.p and self.ultPosicao() == pos_atual: pass
    else: self.p.append(pos_atual)

# Base de Conhecimento

In [21]:
class BD:
    def __init__(self):
        self.conn = sqlite3.connect("base_de_conhecimento")
        self.cursor = self.conn.cursor()
        self.droparTabela(self)
        self.criarTabela(self)

    def imprimirTabela(self):
        query = "select * from base_de_conhecimento"
        query = self.retornarQuery(self, query)
        
        header = ["Tempo", "Conhecimento", "Raciocínio", "Ação"]
        print(tabulate(query, tablefmt="simple", stralign="center", headers=header))

    def inserirConhecimento(self, conhecimento):
        comando = f"insert into base_de_conhecimento (conhecimento) values ('{conhecimento}')"
        self.executarComando(self, comando)
    
    def inserirRaciocinio(self, raciocinio):
        comando = f"insert into base_de_conhecimento (raciocinio) values ('{raciocinio}')"
        self.executarComando(self, comando)
    
    def inserirAcao(self, acao):
        comando = f"insert into base_de_conhecimento (acao) values ('{acao}')"
        self.executarComando(self, comando)
    
    def retornarConhecimento(self):
        query = f"select conhecimento from base_de_conhecimento"
        tabela = self.retornarQuery(self, query)
        conhecimento = []

        for atomo in tabela: 
            if atomo[0]: conhecimento.append(atomo[0])

        return conhecimento
    
    @staticmethod
    def criarTabela(self):
        self.executarComando(self, """
            create table if not exists base_de_conhecimento (
                tempo integer primary key autoincrement,
                conhecimento varchar,
                raciocinio varchar,
                acao varchar
            )
        """)

    @staticmethod
    def droparTabela(self):
        self.executarComando(self, """
            drop table if exists base_de_conhecimento
        """)

    @staticmethod
    def executarComando(self, comando):
        try:
            self.cursor.execute(comando)
            self.conn.commit()
        except sqlite3.Error as erro:
            print(f"Error: {erro}")
    
    @staticmethod
    def retornarQuery(self, query):
        try:
            self.cursor.execute(query)
            query = self.cursor.fetchall()
            return query
        except sqlite3.Error as erro:
            print(f"Erro: {erro}")
            return []

if __name__ == "__main__":
    bd = BD()
    bd.imprimirTabela()


 Tempo    Conhecimento    Raciocínio    Ação
-------  --------------  ------------  ------


# Mapa

In [22]:
class GeradorDeMapa():
    def __init__(self, PreferenciasDeMapa):
        self.linhas = PreferenciasDeMapa.linhas
        self.colunas = PreferenciasDeMapa.colunas
        self.quantidade_wumpus = PreferenciasDeMapa.qtd_wumpus
        self.quantidade_pocos = PreferenciasDeMapa.qtd_pocos
        self.quantidade_ouros = PreferenciasDeMapa.qtd_ouros
        self.taxa_wumpus = self.quantidade_wumpus / (self.linhas * self.colunas - 1 - self.quantidade_pocos - self.quantidade_ouros)
        self.taxa_pocos = self.quantidade_pocos / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_ouros)
        self.taxa_ouros = self.quantidade_ouros / (self.linhas * self.colunas - 1 - self.quantidade_wumpus - self.quantidade_pocos)
        self.status_posicao = {} # {linha}_{coluna}_posicao: ((wumpus, poco, ouro, parede), (fedor, brisa, brilho, impacto, grito))
        self.mapa = []
        self.e_mapa = ElementosDoMapa(self.status_posicao)
        self.gerarMapa()
    
    def retornarStatusPosicao(self):
        return self.status_posicao
    
    def gerarMapa(self):
        self.mapearStatusDePosicao(self)
        for linha in range(-1, self.linhas + 1):
            for coluna in range(-1, self.colunas + 1):
                if self.insercaoValida(self, linha, coluna):
                    self.adicionarWumpusEFedor(self, linha, coluna)
                    self.adicionarPocoEBrisa(self, linha, coluna)
                    self.adicionarOuroEBrilho(self, linha, coluna)
                self.adicionarParedeEImpacto(self, linha, coluna)

    @staticmethod
    def mapearStatusDePosicao(self):
        for linha in range(-1, self.linhas + 1):
            for coluna in range(-1, self.colunas + 1):
                status_default = ((0, 0, 0, 0), (0, 0, 0, 0, 0))
                self.status_posicao[f"{linha}_{coluna}_posicao"] = status_default
    
    @staticmethod
    def insercaoValida(self, linha, coluna):
        posicao_inicial = (linha == 0 and coluna == 0)
        borda = (linha == -1 or linha == self.linhas or coluna == -1 or coluna == self.colunas)
        return not (posicao_inicial or borda)

    @staticmethod
    def adicionarWumpusEFedor(self, linha, coluna):
        pode_adicionar = self.quantidade_wumpus >= 1
        roleta = random() < self.taxa_wumpus
        if pode_adicionar and roleta:
            self.adicionarWumpus(self, linha, coluna)
            self.adicionarFedor(self, linha - 1, coluna)
            self.adicionarFedor(self, linha, coluna - 1)
            self.adicionarFedor(self, linha + 1, coluna)
            self.adicionarFedor(self, linha, coluna + 1)
            self.quantidade_wumpus -= 1
  
    @staticmethod
    def adicionarPocoEBrisa(self, linha, coluna):
        pode_adicionar = self.quantidade_pocos >= 1 and not self.e_mapa.possuiWumpus(linha, coluna)
        roleta = random() < self.taxa_pocos
        if pode_adicionar and roleta:
            self.adicionarPoco(self, linha, coluna)
            self.adicionarBrisa(self, linha - 1, coluna)
            self.adicionarBrisa(self, linha, coluna - 1)
            self.adicionarBrisa(self, linha + 1, coluna)
            self.adicionarBrisa(self, linha, coluna + 1)
            self.quantidade_pocos -= 1
        
    @staticmethod
    def adicionarOuroEBrilho(self, linha, coluna):
        pode_adicionar = self.quantidade_ouros >= 1 and not (self.e_mapa.possuiWumpus(linha, coluna) or self.e_mapa.possuiPoco(linha, coluna))
        roleta = random() < self.taxa_ouros
        if pode_adicionar and roleta:
            self.adicionarOuro(self, linha, coluna)
            self.adicionarBrilho(self, linha, coluna)
            self.quantidade_ouros -= 1
    
    @staticmethod
    def adicionarParedeEImpacto(self, linha, coluna):
        e_borda = (linha == -1 or linha == self.linhas) or (coluna == -1 or coluna == self.colunas)
        if e_borda: 
            self.adicionarParede(self, linha, coluna)
            self.adicionarImpacto(self, linha - 1, coluna)
            self.adicionarImpacto(self, linha, coluna - 1)
            self.adicionarImpacto(self, linha + 1, coluna)
            self.adicionarImpacto(self, linha, coluna + 1)

    @staticmethod
    def adicionarWumpus(self, linha, coluna):
        status = self.status_posicao[f"{linha}_{coluna}_posicao"]
        status = ((status[0][0] + 1, status[0][1], status[0][2], status[0][3]), (status[1]))
        self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
    @staticmethod
    def adicionarPoco(self, linha, coluna):
        status = self.status_posicao[f"{linha}_{coluna}_posicao"]
        status = ((status[0][0], status[0][1] + 1, status[0][2], status[0][3]), (status[1]))
        self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
    @staticmethod
    def adicionarOuro(self, linha, coluna):
        status = self.status_posicao[f"{linha}_{coluna}_posicao"]
        status = ((status[0][0], status[0][1], status[0][2] + 1, status[0][3]), (status[1]))
        self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
    @staticmethod
    def adicionarParede(self, linha, coluna):
        status = self.status_posicao[f"{linha}_{coluna}_posicao"]
        status = ((status[0][0], status[0][1], status[0][2], status[0][3] + 1), (status[1])) 
        self.status_posicao[f"{linha}_{coluna}_posicao"] = status

    @staticmethod
    def adicionarFedor(self, linha, coluna):
        if self.insercaoValida(self, linha, coluna) or (linha == 0 and coluna == 0):
            status = self.status_posicao[f"{linha}_{coluna}_posicao"]
            status = ((status[0]), (status[1][0] + 1, status[1][1], status[1][2], status[1][3], status[1][4]))
            self.status_posicao[f"{linha}_{coluna}_posicao"] = status

    @staticmethod
    def adicionarBrisa(self, linha, coluna):
        if self.insercaoValida(self, linha, coluna) or (linha == 0 and coluna == 0):
            status = self.status_posicao[f"{linha}_{coluna}_posicao"]
            status = ((status[0]), (status[1][0], status[1][1] + 1, status[1][2], status[1][3], status[1][4]))
            self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
    @staticmethod
    def adicionarBrilho(self, linha, coluna):
        status = self.status_posicao[f"{linha}_{coluna}_posicao"]
        status = ((status[0]), (status[1][0], status[1][1], status[1][2] + 1, status[1][3], status[1][4])) 
        self.status_posicao[f"{linha}_{coluna}_posicao"] = status
  
    @staticmethod
    def adicionarImpacto(self, linha, coluna):
        e_linha_externa = linha >= 0 and linha <= self.linhas - 1
        e_coluna_externa = coluna >= 0 and coluna <= self.colunas - 1

        if e_linha_externa and e_coluna_externa:
            status = self.status_posicao[f"{linha}_{coluna}_posicao"]
            status = ((status[0]), (status[1][0], status[1][1], status[1][2], status[1][3] + 1, status[1][4])) 
            self.status_posicao[f"{linha}_{coluna}_posicao"] = status

In [23]:
class ElementosDoMapa():
    def __init__(self, status_posicao):
        self.status_posicao = status_posicao

    def possuiWumpus(self, linha, coluna):
        if self.status_posicao[f"{linha}_{coluna}_posicao"][0][0]:
            return True
        return False
  
    def possuiPoco(self, linha, coluna):
        if self.status_posicao[f"{linha}_{coluna}_posicao"][0][1]:
            return True
        return False
  
    def possuiOuro(self, linha, coluna):
        if self.status_posicao[f"{linha}_{coluna}_posicao"][0][2]:
            return True
        return False
  
    def possuiParede(self, linha, coluna):
        if self.status_posicao[f"{linha}_{coluna}_posicao"][0][3]:
            return True
        return False
  
    def possuiFedor(self, linha, coluna):
        fedor = self.status_posicao[f"{linha}_{coluna}_posicao"][1][0]
        if fedor:
            return True
        return False
    
    def retornarQuantidadeDeFedor(self, linha, coluna):
        fedor = self.status_posicao[f"{linha}_{coluna}_posicao"][1][0]
        return fedor
  
    def possuiBrisa(self, linha, coluna):
        brisa = self.status_posicao[f"{linha}_{coluna}_posicao"][1][1]
        if brisa:
            return brisa
        return False
    
    def retornarQuantidadeDeBrisa(self, linha, coluna):
        brisa = self.status_posicao[f"{linha}_{coluna}_posicao"][1][1]
        return brisa
  
    def possuiBrilho(self, linha, coluna):
        if self.status_posicao[f"{linha}_{coluna}_posicao"][1][2]:
            return True
        return False
  
    def possuiImpacto(self, linha, coluna):
        impacto = self.status_posicao[f"{linha}_{coluna}_posicao"][1][3]
        if impacto:
            return impacto
        return False
    
    def retornarQuantidadeDeImpacto(self, linha, coluna):
        impacto = self.status_posicao[f"{linha}_{coluna}_posicao"][1][3]
        return impacto
  
    def possuiOK(self, linha, coluna):
        status = self.status_posicao[f"{linha}_{coluna}_posicao"]
        for tupla in status:
            for elemento in tupla:
                if elemento != 0:
                    return False
        return True

In [24]:
class Mapa():
    def __init__(self, PreferenciasDeMapa):
        self.linhas = PreferenciasDeMapa.linhas
        self.colunas = PreferenciasDeMapa.colunas
        self.gerador_de_mapa = GeradorDeMapa(PreferenciasDeMapa)
        self.status_posicao = self.gerador_de_mapa.retornarStatusPosicao()
        self.e_mapa = ElementosDoMapa(self.status_posicao)
    
    def retornarStatusDaPosica(self, posicao):
        linha = posicao[0]
        coluna = posicao[1]
        try: return self.status_posicao[f"{linha}_{coluna}_posicao"]
        except:
            print("Posição inválida!")
            return []

    def imprimirMapa(self):
        mapa = self.retornarMapaColorido(self)
        
        print(tabulate(mapa, tablefmt="plain", stralign="center"))
        print(Fore.WHITE) # gambiarra

    @staticmethod
    def retornarMapaColorido(self):
        mapa = []

        init(convert=True, autoreset=True) # colorama
        for linha in range(-1, self.linhas + 1):
            elementos_da_linha = []
            for coluna in range(-1, self.colunas + 1):
                elementos_da_celula = self.retornarElementosDaPosicao(self, linha, coluna)
                elementos_da_linha.append(elementos_da_celula)
            mapa.append(elementos_da_linha)
        deinit() # colorama

        return mapa
    
    @staticmethod
    def retornarElementosDaPosicao(self, linha, coluna):
        elementos_da_celula = ""
        if self.e_mapa.possuiParede(linha, coluna):
            elementos_da_celula += (Fore.MAGENTA + "parede")
        if self.e_mapa.possuiImpacto(linha, coluna):
            elementos_da_celula += (Fore.MAGENTA + str(self.e_mapa.retornarQuantidadeDeImpacto(linha, coluna)) + "impacto")
        if self.e_mapa.possuiWumpus(linha, coluna):
            elementos_da_celula += (Fore.GREEN + "wumpus")
        if self.e_mapa.possuiPoco(linha, coluna):
            elementos_da_celula += (Fore.RED + "poco")
        if self.e_mapa.possuiOuro(linha, coluna):
            elementos_da_celula += (Fore.YELLOW + "ouro")
        if self.e_mapa.possuiFedor(linha, coluna):
            elementos_da_celula += (Fore.GREEN + str(self.e_mapa.retornarQuantidadeDeFedor(linha, coluna)) + "fedor")
        if self.e_mapa.possuiBrisa(linha, coluna):
            elementos_da_celula += (Fore.RED + str(self.e_mapa.retornarQuantidadeDeBrisa(linha, coluna)) + "brisa")
        if self.e_mapa.possuiBrilho(linha, coluna):
            elementos_da_celula += (Fore.YELLOW + "brilho")
        if self.e_mapa.possuiOK(linha, coluna):
            elementos_da_celula += (Fore.WHITE + "ok")
        
        return elementos_da_celula

# Atuador

In [25]:
class Atuador():
  def __init__(self, Solver, BD, Pilha, InfoDoAgente):
    self.bd = BD
    self.p = Pilha # Busca em profundidade
    self.info = InfoDoAgente
    self.s = Solver

  def Atuar(self, acao):
    if acao == "saindo_da_caverna":
      if self.sairDaCaverna(self): 
        print("saiu da caverna")
        self.info.status = "fim"
        self.bd.inserirAcao("sair_da_caverna")
      else: 
        self.caminharParaSaida(self)      
    elif acao == "pegar_ouro": 
      self.pegarOuro(self)
    elif acao == "frente": 
      self.avancar(self)
    elif acao == "esquerda": 
      self.girarParaEsquerda(self)
      self.bd.inserirAcao("avancar")
      self.avancar(self)
    elif acao == "direita":
      self.girarParaDireita(self)
      self.bd.inserirAcao("avancar")
      self.avancar(self)
    else: 
      self.caminharParaSaida(self)
    
  @staticmethod
  def sairDaCaverna(self):
    if self.info.posicaoAtual() == (0, 0):
      self.info.desempenho -= 1
      return True
    else: return False
  
  @staticmethod
  def caminharParaSaida(self):
    try:
      self.p.pop() # Posição atual
      ultima_posicao = self.p.ultPosicao()
      linha = ultima_posicao[0]
      coluna = ultima_posicao[1]

      if linha < self.info.linha: self.girarParaOrientacao(self, 90)
      elif linha > self.info.linha: self.girarParaOrientacao(self, 270)
      elif coluna < self.info.coluna: self.girarParaOrientacao(self, 180)
      elif coluna > self.info.coluna: self.girarParaOrientacao(self, 0)

      self.avancar(self)
    except: 
      print("saiu da caverna")
      self.info.status_agente = "fim"

  @staticmethod
  def pegarOuro(self):
    self.s.adicionarRegra(f"{self.info.linha}_{self.info.coluna}_ouro_pego")
    self.info.desempenho += 1000
  
  @staticmethod
  def avancar(self):
    if self.info.orientacao == 0: self.info.coluna += 1
    elif self.info.orientacao == 90: self.info.linha -= 1
    elif self.info.orientacao == 180: self.info.coluna -= 1
    elif self.info.orientacao == 270: self.info.linha += 1

    self.info.desempenho -= 1

  @staticmethod
  def girarParaOrientacao(self, orientacao):
    diferenca = orientacao - self.info.orientacao

    if diferenca == 0: pass
    elif diferenca == 90: 
      self.girarParaEsquerda(self)
    elif diferenca == -90: 
      self.girarParaDireita(self)
    elif diferenca == 180:
      self.girarParaEsquerda(self)
      self.girarParaEsquerda(self)
    elif diferenca == -180:
      self.girarParaEsquerda(self)
      self.girarParaEsquerda(self)
    elif diferenca == 270:
      self.girarParaDireita(self)
    elif diferenca == -270:
      self.girarParaEsquerda(self)

  @staticmethod
  def girarParaEsquerda(self):
    self.info.incrementarOrientacao()
    self.info.desempenho -= 1
  
  @staticmethod
  def girarParaDireita(self):
    self.info.decrementarOrientacao()
    self.info.desempenho -= 1

# Solver

In [26]:
class Solver():
  def __init__(self, tamanho_do_mapa, bd):
    self.glucose = Glucose3()
    self.linhas = tamanho_do_mapa[0]
    self.colunas = tamanho_do_mapa[1]
    self.bd = bd
    self.mapeamento = {}
    self.mapeamento_reverso = {}
    self.index = 1
    self.atomo = ""
    self.atomo_cima = ""
    self.atomo_esquerda = ""
    self.atomo_direita = ""
    self.atomo_baixo = ""
    self.mapear(self)
    self.regrasDoMapa(self)
  
  def perguntarRegra(self, regra):
    # Verdade caso não aja conflito ou por facuidade
    try: regra = self.mapeamento[regra]
    except: 
      self.mapeamento[f"{regra}"] = self.index
      self.mapeamento[f"- {regra}"] = -self.index
      self.mapeamento_reverso[self.index] = f"{regra}"
      self.mapeamento_reverso[-self.index] = f"- {regra}"

      self.index += 1
      regra = self.mapeamento[regra]

    if self.glucose.solve(assumptions=[regra]): 
      return True
    return False
  
  def perguntarRegraComCerteza(self, regra):
    # Igual perguntarRegra(), porém retorna False por vacuidade
    if "-" in regra:
      regra_n = self.perguntarRegra(f"{regra}") 
      regra = self.perguntarRegra(f"{regra[2:]}")
      regra = regra_n and not regra 
    else:
      regra_n = self.perguntarRegra(f"- {regra}")
      regra = self.perguntarRegra(f"{regra}")
      regra = regra and not regra_n

    if regra: return True
    else: return False

  def adicionarRegra(self, regra):
    self.mapeamento[f"{regra}"] = self.index
    self.mapeamento[f"- {regra}"] = -self.index
    self.mapeamento_reverso[self.index] = f"{regra}"
    self.mapeamento_reverso[-self.index] = f"- {regra}"
    self.index += 1

    self.atomo = self.mapeamento[f"{regra}"]
    self.eVerdade(self)

  def naoPossuiFedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eFalso(self)

    self.naoPossui(self, linha, coluna, "wumpus")
    
  def naoPossuiBrisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eFalso(self)

    self.naoPossui(self, linha, coluna, "poco")
    
  def possui1Fedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eVerdade(self)

    self.possui1Verdadeiro(self, linha, coluna, "wumpus")

  def possui1Brisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eVerdade(self)

    self.possui1Verdadeiro(self, linha, coluna, "poco")
     
  def possui2Fedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eVerdade(self)

    self.possui2Verdadeiro(self, linha, coluna, "wumpus")

  def possui2Brisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eVerdade(self)

    self.possui2Verdadeiro(self, linha, coluna, "poco")

  def possui3Fedor(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_fedor"]
    self.eVerdade(self)

    self.possui3Verdadeiro(self, linha, coluna, "wumpus")

  def possui3Brisa(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brisa"]
    self.eVerdade(self)
    
    self.possui3Verdadeiro(self, linha, coluna, "poco")

  def naoPossuiBrilho(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brilho"]
    self.eFalso(self)

    self.atomo = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.eFalso(self)
  
  def possuiBrilho(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_brilho"]
    self.eVerdade(self)

    self.atomo = self.mapeamento[f"{linha}_{coluna}_ouro"]
    self.eVerdade(self)
  
  def naoPossuiImpacto(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.eFalso(self)
    
    self.naoPossui(self, linha, coluna, "parede")

  def possui1Impacto(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.eVerdade(self)

    self.possui1Verdadeiro(self, linha, coluna, "parede")

    self.impactosAdjacentesVertical(self, linha, coluna)

    self.impactosAdjacentesHorizontal(self, linha, coluna)

  def possui2Impacto(self, linha, coluna):
    self.atomo = self.mapeamento[f"{linha}_{coluna}_impacto"]
    self.eVerdade(self)

    self.possui2Verdadeiro(self, linha, coluna, "parede")
    
    self.impactosAdjacentesVertical(self, linha, coluna)

    self.impactosAdjacentesHorizontal(self, linha, coluna)

  @staticmethod
  def regrasDoMapa(self):
    for linha in range(0, self.linhas):
      for coluna in range(0, self.colunas):
        atomo1 = self.mapeamento[f"{linha}_{coluna}_wumpus"]
        atomo2 = self.mapeamento[f"{linha}_{coluna}_poco"]
        atomo3 = self.mapeamento[f"{linha}_{coluna}_ouro"]

        self.glucose.add_clause([-atomo1, -atomo2])
        self.glucose.add_clause([-atomo1, -atomo3])
        self.glucose.add_clause([-atomo2, -atomo3])

    for linha in [-1, 0, self.linhas - 1, self.linhas]:
      for coluna in range(-1, self.colunas + 1):
        try: atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]
        except: 
          self.mapearInstancia(self, linha, coluna, "parede")
          atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]

        try: atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]
        except: 
          self.mapearInstancia(self, linha, coluna, "impacto")
          atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]

        self.glucose.add_clause([-atomo1, -atomo2])

    for linha in range(-1, self.linhas):
      for coluna in [-1, 0, self.linhas - 1, self.linhas]:
        try: atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]
        except: 
          self.mapearInstancia(self, linha, coluna, "parede")
          atomo1 = self.mapeamento[f"{linha}_{coluna}_parede"]

        try: atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]
        except: 
          self.mapearInstancia(self, linha, coluna, "impacto")
          atomo2 = self.mapeamento[f"{linha}_{coluna}_impacto"]

        self.glucose.add_clause([-atomo1, -atomo2])
    
    self.glucose.add_clause([self.mapeamento[f"-1_0_parede"]])
    self.glucose.add_clause([self.mapeamento[f"0_-1_parede"]])

    # Remédio para o bug, agente acha que existe paredes internar
    # Não há parede internas
    for linha in range(0, self.linhas - 1):
      for coluna in range(0, self.colunas - 1): 
        self.mapearInstancia(self, linha, coluna, "parede")
        self.glucose.add_clause([self.mapeamento[f"- {linha}_{coluna}_parede"]])
    
  @staticmethod
  def mapearInstancia(self, linha, coluna, objeto):
    try: self.mapeamento[f"{linha}_{coluna}_{objeto}"]
    except:
      self.mapeamento[f"{linha}_{coluna}_{objeto}"] = self.index
      self.mapeamento[f"- {linha}_{coluna}_{objeto}"] = -self.index
      self.mapeamento_reverso[self.index] = f"{linha}_{coluna}_{objeto}"
      self.mapeamento_reverso[-self.index] = f"- {linha}_{coluna}_{objeto}"

      self.index += 1

  @staticmethod
  def mapear(self):
    for linha in range(0, self.linhas):
      for coluna in range(0, self.colunas):
        self.mapearInstancia(self, linha, coluna, "wumpus")
        self.mapearInstancia(self, linha, coluna, "poco")
        self.mapearInstancia(self, linha, coluna, "ouro")
        self.mapearInstancia(self, linha, coluna, "fedor")
        self.mapearInstancia(self, linha, coluna, "brisa")
        self.mapearInstancia(self, linha, coluna, "brilho")
        self.mapearInstancia(self, linha, coluna, "impacto")
    
    for linha in [0, self.linhas - 1]:
      for coluna in range(0, self.colunas):
        self.mapearInstancia(self, linha, coluna, "impacto")

    for linha in range(0, self.linhas):
      for coluna in [0, self.colunas - 1]:
        self.mapearInstancia(self, linha, coluna, "impacto")
    
    for linha in [-1, self.linhas]:
      for coluna in range(-1, self.colunas + 1):
        self.mapearInstancia(self, linha, coluna, "parede")

    for linha in range(-1, self.linhas + 1):
      for coluna in [-1, self.colunas]:
        self.mapearInstancia(self, linha, coluna, "parede")
    
    self.mapeamento[f"verdade"] = self.index
    self.mapeamento_reverso[self.index] = f"verdade"

    self.glucose.add_clause([self.index])

    self.mapeamento[f"falso"] = -self.index
    self.mapeamento_reverso[-self.index] = f"falso"

  def imprimirMapeamento(self):
    print("mapeamento[posição] -> index")
    print(self.mapeamento)
    print("mapeamento_reverso[index] -> posição")
    print(self.mapeamento_reverso)
  
  @staticmethod
  def inserirConhecimento(self, conhecimento):
    self.bd.inserirConhecimento(f"{self.mapeamento_reverso[conhecimento]}")

  @staticmethod
  def inserirRaciocinio(self, raciocinio):
    regra = ""
    for atomo in raciocinio:
      regra += f"{self.mapeamento_reverso[atomo]}, "
    regra = regra[0:-2]
    self.bd.inserirRaciocinio(regra)
  
  def resolverSolve(self):
    self.glucose.solve()

  def retornarSolve(self):
    self.glucose.get_model()
  
  def imprimirSolve(self):
    print(self.glucose.get_model())

  # Equivalências Lógicas

  @staticmethod
  def eVerdade(self):
    self.glucose.add_clause([self.atomo])

    self.inserirConhecimento(self, self.atomo)

  @staticmethod
  def eFalso(self):
    self.glucose.add_clause([-self.atomo])

    self.inserirConhecimento(self, -self.atomo)
  
  @staticmethod
  def naoPossui(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima])
    self.glucose.add_clause([-self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita])

    self.inserirConhecimento(self, -self.atomo_cima)
    self.inserirConhecimento(self, -self.atomo_esquerda)
    self.inserirConhecimento(self, -self.atomo_baixo)
    self.inserirConhecimento(self, -self.atomo_direita)

  @staticmethod
  def possui1Verdadeiro(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_baixo, -self.atomo_direita])
  
  @staticmethod
  def possui2Verdadeiro(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_baixo, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])

  @staticmethod
  def possui3Verdadeiro(self, linha, coluna, objeto):
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, objeto)
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_{objeto}"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, objeto)
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_{objeto}"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, objeto)
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_{objeto}"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, objeto)
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_{objeto}"]

    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])
    self.glucose.add_clause([self.atomo_cima, self.atomo_esquerda])
    self.glucose.add_clause([self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_cima, self.atomo_direita])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_baixo])
    self.glucose.add_clause([self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([self.atomo_baixo, self.atomo_direita])

    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_baixo, -self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_esquerda])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_cima, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_baixo])
    self.inserirRaciocinio(self, [self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [self.atomo_baixo, self.atomo_direita])
  
  @staticmethod
  def impactosAdjacentesHorizontal(self, linha, coluna):
    # ((p&e)#(p&d))_impacto>((c&-b)#(-c&b))_parede
    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, "parede")
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_parede"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_impacto"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, "impacto")
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_impacto"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, "parede")
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_parede"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_impacto"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, "impacto")
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_impacto"]

    self.glucose.add_clause([-self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.glucose.add_clause([-self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])  

    self.inserirRaciocinio(self, [-self.atomo_esquerda, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_esquerda, -self.atomo_cima, -self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_direita, self.atomo_cima, self.atomo_baixo])
    self.inserirRaciocinio(self, [-self.atomo_direita, -self.atomo_cima, -self.atomo_baixo])
  
  @staticmethod
  def impactosAdjacentesVertical(self, linha, coluna):

    try: self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_impacto"]
    except: 
      self.mapearInstancia(self, linha - 1, coluna, "impacto")
      self.atomo_cima = self.mapeamento[f"{linha - 1}_{coluna}_impacto"]

    try: self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]
    except:
      self.mapearInstancia(self, linha, coluna - 1, "parede")
      self.atomo_esquerda = self.mapeamento[f"{linha}_{coluna - 1}_parede"]

    try: self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_impacto"]
    except:
      self.mapearInstancia(self, linha + 1, coluna, "impacto")
      self.atomo_baixo = self.mapeamento[f"{linha + 1}_{coluna}_impacto"]

    try: self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]
    except:
      self.mapearInstancia(self, linha, coluna + 1, "parede")
      self.atomo_direita = self.mapeamento[f"{linha}_{coluna + 1}_parede"]

    self.glucose.add_clause([-self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.glucose.add_clause([-self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita]) 

    self.inserirRaciocinio(self, [-self.atomo_cima, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_cima, -self.atomo_esquerda, -self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_baixo, self.atomo_esquerda, self.atomo_direita])
    self.inserirRaciocinio(self, [-self.atomo_baixo, -self.atomo_esquerda, -self.atomo_direita])   

# Agente

In [27]:
class Agente():
  def __init__(self, s, BD):
    self.s = s
    self.bd = BD
    self.info = InfoDoAgente()
    self.p = Pilha()
    self.proxima_direcao = []
    self.atuador = Atuador(self.s, self.bd, self.p, self.info)

  def retornarInformacoesDoAgente(self): return self.info
  def atualizarPilha(self, Pilha): self.p = Pilha
  
  # TODO fazer objeto percepcao
  def Agente(self, percepcao):
    self.inferir(self, percepcao)
    acao = self.escolherAcao(self)
    return acao
  
  @staticmethod
  def inferir(self, percepcao):
    fedor = percepcao[1][0]
    brisa = percepcao[1][1]
    brilho = percepcao[1][2]
    impacto = percepcao[1][3]

    if fedor == 0: self.s.naoPossuiFedor(self.info.linha, self.info.coluna)
    elif fedor == 1: self.s.possui1Fedor(self.info.linha, self.info.coluna)
    elif fedor == 2: self.s.possui2Fedor(self.info.linha, self.info.coluna)
    elif fedor == 3: self.s.possui3Fedor(self.info.linha, self.info.coluna)
    
    if brisa == 0: self.s.naoPossuiBrisa(self.info.linha, self.info.coluna)
    elif brisa == 1: self.s.possui1Brisa(self.info.linha, self.info.coluna)
    elif brisa == 2: self.s.possui2Brisa(self.info.linha, self.info.coluna)
    elif brisa == 3: self.s.possui3Brisa(self.info.linha, self.info.coluna)
    
    if impacto == 1: self.s.possui1Impacto(self.info.linha, self.info.coluna)
    elif impacto == 2: self.s.possui2Impacto(self.info.linha, self.info.coluna)
    else: self.s.naoPossuiImpacto(self.info.linha, self.info.coluna)
    
    if brilho == 0: self.s.naoPossuiBrilho(self.info.linha, self.info.coluna)
    elif brilho == 1: self.s.possuiBrilho(self.info.linha, self.info.coluna)

    posicao_explorada = f"{self.info.linha}_{self.info.coluna}_explorado"
    self.s.adicionarRegra(posicao_explorada)  
  
  def imprimirMapa(self):
      mapa = []
      init() # colorama
      for linha in range(-1, 5):
        elementos_da_linha = []
        for coluna in range(-1, 5):
          elementos_da_celula = ""

          if linha == self.info.linha and coluna == self.info.coluna:
            elementos_da_celula += (Fore.WHITE + f"agente{self.info.orientacao}")

          if self.s.perguntarRegra(f"{linha}_{coluna}_wumpus"):
            elementos_da_celula += (Fore.GREEN + "wumpus")
          if self.s.perguntarRegra(f"{linha}_{coluna}_poco"):
            elementos_da_celula += (Fore.RED + "poco")
          if self.s.perguntarRegra(f"{linha}_{coluna}_ouro"):
            elementos_da_celula += (Fore.YELLOW + "ouro")
          if self.s.perguntarRegra(f"{linha}_{coluna}_parede"):
            elementos_da_celula += (Fore.MAGENTA + "parede")
          if self.s.perguntarRegra(f"{linha}_{coluna}_fedor"):
            elementos_da_celula += (Fore.GREEN + "fedor")
          if self.s.perguntarRegra(f"{linha}_{coluna}_brisa"):
            elementos_da_celula += (Fore.RED + "brisa")
          if self.s.perguntarRegra(f"{linha}_{coluna}_brilho"):
            elementos_da_celula += (Fore.YELLOW + "brilho")
          if self.s.perguntarRegra(f"{linha}_{coluna}_impacto"):
            elementos_da_celula += (Fore.MAGENTA + "impacto")
          
          if elementos_da_celula == "": elementos_da_celula += (Fore.WHITE + "ok") 
          
          elementos_da_linha.append(elementos_da_celula)
        mapa.append(elementos_da_linha)
      deinit() # colorama
      print(tabulate(mapa, tablefmt="plain", stralign="center"))
      print(Fore.WHITE) # bug - remover fará com que todas os outputs fiquem coloridos
  
  @staticmethod
  def escolherAcao(self):
    explorar = self.podeExplorar(self)

    if self.podeSairDaCaverna(self): return "saindo_da_caverna"
    elif self.temOuro(self): return "pegar_ouro"
    elif explorar: return explorar
    else: return "retornar"
  
  @staticmethod
  def podeSairDaCaverna(self): 
    if self.info.desempenho > 0: return True
    elif self.info.desempenho < 0 and self.p.len() == 1 \
      and not self.proxima_direcao: return True
    else: return False

  @staticmethod
  def temOuro(self):
    regra_tem_ouro = f"{self.info.linha}_{self.info.coluna}_ouro"
    ouro = self.s.perguntarRegraComCerteza(regra_tem_ouro)
    regra_foi_pego = f"{self.info.linha}_{self.info.coluna}_ouro_pego"
    pego = self.s.perguntarRegraComCerteza(regra_foi_pego)
    
    if ouro and not pego: return True
    else: return False
  
  @staticmethod
  def podeExplorar(self):
    status_adjacentes = self.retornarStatusAdjacentes(self)
    # print(status_adjacentes)
    proxima_direcao = self.retornarProximaDirecao(self, status_adjacentes)
    return proxima_direcao
    
  @staticmethod
  def retornarStatusAdjacentes(self):
    status_adjacentes = {
      # direcao: (pode_avancar, foi_explorado)
    }

    for direcao in ["frente", "esquerda", "atras", "direita"]:
      pode_avancar = False
      foi_explorado = True

      if self.podeAvancar(self): pode_avancar = True

      linha, coluna = self.info.posicaoAFrente()
      if not self.foiExplorado(self, linha, coluna): foi_explorado = False
      
      status_adjacentes[direcao] = (pode_avancar, foi_explorado)
      self.info.incrementarOrientacao()

    return status_adjacentes

  @staticmethod
  def retornarProximaDirecao(self, status_adjacentes):
    proxima_direcao = [] # Prioridade: (True, False)
    
    for direcao in ["frente", "esquerda", "atras", "direita"]:
      if status_adjacentes[direcao] == (True, False):
        proxima_direcao.append(direcao) 
    
    self.proxima_direcao = proxima_direcao # usado em podeSairDaCaverna()

    if proxima_direcao: return choice(proxima_direcao)
    return []

  @staticmethod
  def podeAvancar(self):
    linha, coluna = self.info.posicaoAFrente()

    try:
      wumpus = self.s.perguntarRegra(f"{linha}_{coluna}_wumpus")
      poco = self.s.perguntarRegra(f"{linha}_{coluna}_poco")
      parede = self.s.perguntarRegra(f"{linha}_{coluna}_parede")

      return not (wumpus or poco or parede)
    except: return False
  
  @staticmethod
  def foiExplorado(self, linha, coluna):
    regra = f"{linha}_{coluna}_explorado"
    explorado = self.s.perguntarRegraComCerteza(regra)
    # explorado = not (self.s.perguntarRegra(f"{linha}_{coluna}_explorado") and self.s.perguntarRegra(f"- {linha}_{coluna}_explorado"))
    return explorado

# Mundo de Wumpus

In [28]:
class MundoDeWumpus():
  def __init__(self, PreferenciasDeMapa, BD):
    self.mapa = Mapa(PreferenciasDeMapa)
    self.bd = BD
    self.s = Solver(PreferenciasDeMapa.tamanho(), self.bd)
    self.p = Pilha() # Busca em profundidade
    self.agente = Agente(self.s, self.bd)
    self.atuador = Atuador(self.s, self.bd, self.p, self.agente.info)
  
  def resolverMundoDeWumpus(self):
    info = self.agente.retornarInformacoesDoAgente()

    #while info.status == "explorando":
    if info.status == "explorando":
      self.p.add(info.posicaoAtual())
      self.agente.atualizarPilha(self.p)

      percepcao = self.receberSensor(self)
      acao = self.agente.Agente(percepcao)
      self.bd.inserirAcao(acao)

      print(f"posicao: {info.posicaoAtual()}") 
      print(f"percepcao: {percepcao}") 
      print(f"desempenho: {info.desempenho} status: {info.status}")
      print(f"ação: {acao}")
      print(f"pilha_de_exploracao: {self.p.get()}")

      self.atuador.Atuar(acao)

      self.validarVida(self, percepcao)
  
  def imprimirMapa(self):
    self.mapa.imprimirMapa()
  
  def imprimirMapaDoConhecimento(self):
    self.agente.imprimirMapa()
  
  def imprimirBaseDeConhecimento(self):
    self.bd.imprimirTabela()
  
  @staticmethod
  def validarVida(self, percepcao):
    wumpus = percepcao[0][0]
    poco = percepcao[0][1]
    parede = percepcao[0][3]

    if wumpus or poco or parede: 
      self.agente.info.status = "morto"
      print("Agente morto")
      exit(1) 
  
  @staticmethod
  def receberSensor(self):
    pos_atual = self.agente.info.posicaoAtual()
    return self.mapa.retornarStatusDaPosica(pos_atual)  

# Main

In [ ]:
pref_mapa = PreferenciasDeMapa(
              linhas = 4,
              colunas = 4, 
              qtd_wumpus = 1, 
              qtd_pocos = 1, 
              qtd_ouros = 2
            )
bd = BD()
mw = MundoDeWumpus(pref_mapa, bd)

while(input() == "" and mw.agente.info.status != "fim"):
  mw.resolverMundoDeWumpus()
  mw.imprimirMapa()
  mw.imprimirMapaDoConhecimento()
  # bd.imprimirTabela()